<a href="https://colab.research.google.com/github/EvgenyEsin/Python_libraries_for_DS/blob/main/Python_libraries_for_DS_DZ_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ к семинару 9

Цель задания: Исследовать влияние различных методов понижения размерности на качество классификации текстовых данных.

Датасет: Набор данных новостных статей
(датасет '20 Newsgroups' доступный в sklearn.datasets).

Задачи:

1. Загрузите датасет '20 Newsgroups' из sklearn.

2. Проведите предобработку данных (очистка текста, удаление стоп-слов, векторизация с использованием TF-IDF).

3. Примените к полученным векторам TF-IDF следующие методы понижения размерности:
— PCA (Principal Component Analysis)
— t-SNE (t-distributed Stochastic Neighbor Embedding)
— UMAP (Uniform Manifold Approximation and Projection).

4. После понижения размерности данных используйте любой метод машинного обучения для классификации новостей по темам.

5. Сравните качество классификации для каждого метода понижения размерности. Используйте метрики точности и F1-меру.

6. Визуализируйте двумерное представление данных для каждого метода понижения размерности, чтобы оценить, как алгоритмы справляются с сепарацией классов.

7. Напишите отчёт, в котором обсудите, какой метод понижения размерности оказал наиболее значительное влияние на качество классификации и почему.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
!pip install umap-learn
from umap import UMAP
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.9 MB/s eta 0:00:00


Загрузка данных

мы будем использовать встроенный загрузчик наборов данных для выборки «The 20 newsgroups» из scikit-learn. Иначе, выборку можно загрузить вручную с вэб сайта, использовать функцию sklearn.datasets.load_files и указав папку «20news-bydate-train» для сохранения распакованного архива.
Чтобы первый пример быстрее исполнялся, мы будем работать только с частью нашего набора данных, разбитой на 4 категории из 20 возможных:

In [13]:
df = fetch_20newsgroups(data_home=None, subset='train', categories=None, shuffle=True, random_state=42, remove=(), download_if_missing=True, return_X_y=False, n_retries=3, delay=1.0)

Мы можем загрузить список файлов, совпадающих с нужными категориями, как показано ниже:

Возвращаемый набор данных — это scikit-learn совокупность: одномерный контейнер с полями, которые могут интерпретироваться как ключи в словаре python (dict keys), проще говоря — как признаки объекта (object attributes). Например, target_names содержит список названий запрошенных категорий:

In [25]:
from sklearn.datasets import fetch_20newsgroups
cats = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
newsgroups_train = fetch_20newsgroups(subset='train', categories=cats, shuffle=True, random_state=42)
list(newsgroups_train.target_names)

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

Сами файлы загружаются в память как атрибут data. Вы также можете ссылаться на названия файлов:

In [26]:
newsgroups_train.filenames.shape

(2257,)

In [27]:
newsgroups_train.target.shape

(2257,)

Давайте выведем на печать первые строки из первого загруженного файла:

In [29]:
print("\n".join(newsgroups_train.data[0].split("\n")[:3]))

print(newsgroups_train.target_names[newsgroups_train.target[0]])

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
comp.graphics


Алгоритмы для обучения с учителем (контролируемого обучения) требуют, чтобы у каждого документа в обучающей выборке была помета определенной категории. В нашем случае, категория — это название новостной выборки, которая «случайно» оказывается названием папки, содержащей характерные документы.
Для увеличения скорости и эффективного использования памяти, scikit-learn загружает целевой атрибут как массив целых чисел, который соответствует индексу названия категории из списка target_names. Индекс категории каждой выборки хранится в атрибуте target:

In [30]:
newsgroups_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2])

Извлечение характерных признаков из текстовых файлов

Чтобы использовать машинное обучение на текстовых документах, первым делом, нужно перевести текстовое содержимое в числовой вектор признаков.
«Мешок слов» (набор слов)

Наиболее интуитивно понятный способ сделать описанное выше преобразование — это представить текст в виде набора слов:
приписать уникальный целочисленный индекс каждому слову, появляющемуся в документах в обучающей выборке (например, построив словарь из слов с целочисленными индексами).
для каждого документа #i посчитать количество употреблений каждого слова w и сохранить его (количество) в X[i, j]. Это будет значение признака #j, где j — это индекс слова w в словаре.

Представление «мешок слов» подразумевает, что n_features — это некоторое количество уникальных слов в корпусе. Обычно, это количество превышает 100000.
Если n_samples == 10000, то Х, сохраненный как массив numpy типа float32, потребовал бы 10000 x 100000 x 4 bytes = 4GB оперативной памяти (RAM), что едва осуществимо в современным компьютерах.
К счастью, большинство значений в X являются нулями, поскольку в одном документе используется менее чем пара сотен уникальных слов. Поэтому «мешок слов» чаще всего является высоко размерным разреженным набором данных. Мы можем сэкономить много свободной оперативки, храня в памяти только лишь ненулевые части векторов признаков.
Матрицы scipy.sparse — это структуры данных, которые именно это и делают — структурируют данные. В scikit-learn есть встроенная поддержка этих структур.

Токенизация текста с scikit-learn

Предобработка текста, токенизация и отфильтровывание стоп-слов включены в состав высоко уровневого компонента, который позволяет создать словарь характерных признаков и перевести документы в векторы признаков:

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(newsgroups_train.data)
X_train_counts.shape

(2257, 35788)

CountVectorizer поддерживает подсчет N-грам слов или последовательностей символов. Векторизатор строит словарь индексов признаков:

In [34]:
count_vect.vocabulary_.get(u'algorithm')

4690

Значение индекса слова в словаре связано с его частотой употребления во всем обучающем корпусе.

От употреблений к частотности

Подсчет словоупотреблений — это хорошее начало, но есть проблема: в длинных документах среднее количество словоупотреблений будет выше, чем в коротких, даже если они посвящены одной теме.
Чтобы избежать этих потенциальных несоответствий, достаточно разделить количество употреблений каждого слова в документе на общее количество слов в документе. Этот новый признак называется tf — Частота термина.
Следующее уточнение меры tf — это снижение веса слова, которое появляется во многих документах в корпусе, и отсюда является менее информативным, чем те, которые используются только в небольшой части корпуса. Примером низко ифнормативных слов могут служить служебные слова, артикли, предлоги, союзы и т.п.
Это снижение называется tf–idf, что значит “Term Frequency times Inverse Document Frequency” (обратная частота термина).
Обе меры tf и tf–idf могут быть вычислены следующим образом:

In [35]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

В примере кода, представленного выше, мы сначала используем метод fit(..), чтобы прогнать наш алгоритм оценки на данных, а потом — метод transform(..), чтобы преобразовать нашу числовую матрицу к представлению tf-idf. Эти два шага могут быть объединены и дадут тот же результат на выходе, но быстрее, что можно сделать с помощью пропуска излишней обработки. Для этого нужно использовать метод fit_transform(..), как показано ниже:

In [36]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

## Обучение классификатора


Теперь, когда мы выделили признаки, можно обучать классификатор предсказывать категорию текста. Давайте начнем с Наивного Байесовского классификатора, который станет прекрасной отправной точкой для нашей задачи. scikit-learn включает в себя несколько вариантов этого классификатора. Самый подходящий для подсчета слов — это его поли номинальный вариант:

In [38]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, newsgroups_train.target)

Чтобы мы могли попытаться предсказать результаты на новом документе, нужно извлечь фичи (характерные признаки), используя почти такую же последовательность как и ранее. Разница состоит в том, используется метод transform вместо fit_transform из transformers, так как они уже были применены к нашей обучающей выборке:

In [41]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
  print('%r => %s' % (doc, newsgroups_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


## Создание конвейерной обработки

Чтобы с цепочкой vectorizer => transformer => classifier было проще работать, в scikit-learn есть класс Pipeline, который функционирует как составной (конвейерный) классификатор:

In [44]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                     ])

Теперь обучим модель с помощью всего 1 команды:

In [46]:
text_clf = text_clf.fit(newsgroups_train.data, newsgroups_train.target)

## Оценка производительности при работе на тестовой выборке

Оценка точности прогноза модели достаточно проста:

In [48]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',
    categories=cats, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.8348868175765646

Например, мы получили 83% точности. Давайте посмотрим, можем ли мы улучшить этот результат с помощью линейного метода опорных векторов (support vector machine (SVM)). Этот метод обычно считается лучшим из алгоритмов классификации текста (хотя, он немного медленнее чем наивный Байес). Мы можем сменить обучающуюся модель просто подсоединив другой объект классификации в наш конвейер:

In [52]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42)),])
_ = text_clf.fit(newsgroups_train.data, newsgroups_train.target)
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.9114513981358189

scikit-learn также предоставляет утилиты для более детального анализа результатов:

In [53]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,
    target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.81      0.88       319
         comp.graphics       0.87      0.98      0.92       389
               sci.med       0.96      0.88      0.92       396
soc.religion.christian       0.89      0.96      0.92       398

              accuracy                           0.91      1502
             macro avg       0.92      0.91      0.91      1502
          weighted avg       0.92      0.91      0.91      1502



In [54]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[259,  10,  12,  38],
       [  3, 380,   2,   4],
       [  6,  36, 349,   5],
       [  5,  10,   2, 381]])

Как и ожидалось, матрица неточностей показывает, что тексты из выборки newsgroups об атеизме и христианстве модель чаще путает друг с другом, нежели чем с текстами про компьютерную графику.

Настройка параметров для использования grid search

Некоторые параметры мы уже посчитали, такие как use_idf в функции TfidfTransformer. Классификаторы, как правило, также имеют много параметров, например, MultinomialNB включает в себя коэффициент сглаживания alpha, а SGDClassifier имеет штрафной параметр alpha (метод штрафных функций), настраиваемую потерю и штрафные члены в целевой функции (см. раздел документации или используйте функцию подсказки Python, чтобы получить дополнительную информацию).
Вместо поиска параметров различных компонентов в цепи, можно запустить поиск (методом полного перебора ) лучших параметров в сетке возможных значений. Мы опробовали все классификаторы на словах или биграммах, с или без idf, с штрафными параметрами 0,01 и 0,001 для SVM (метода опорных векторов):

In [59]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),
}

Очевидно, подобный поиск методом полного перебора может быть ресурсозатратным. Если в нашем распоряжении есть множество ядер процессора, мы можем запустить grid search, чтобы попробовать все комбинации параметров параллельно с параметром n_jobs. Если мы зададим этому параметру значение -1, grid search определит, как много ядер установлено, и задействует их все:

In [60]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)


Экземпляр grid search ведет себя как обычная модель scikit-learn. Давайте запустим поиск на малой части обучающей выборки, чтобы увеличить скорость обработки:

In [62]:
gs_clf = gs_clf.fit(newsgroups_train.data[:400], newsgroups_train.target[:400])

В результате применения метода fit на объекте GridSearchCV мы получим классификатор, который можно использовать для выполнения функции predict:

In [68]:
newsgroups_train.target[gs_clf.predict(['God is love'])]

array([3])